In [1]:
from collections import defaultdict
from itertools import combinations

In [2]:
data = open("input/12").read().splitlines()

In [3]:
grid = {}
for r, row in enumerate(data):
    for c, elem in enumerate(row):
        grid[(r, c)] = elem

In [4]:
def bfs_find_paths(pos, idx):
    plot_key = grid.get(pos) + "_" + str(idx)
    plots = set()
    queue = [pos]
    visited = set()
    while queue:
        current_pos = queue.pop(0)
        current_value = grid.get(current_pos)
        plots.add(current_pos)
        for direction in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbour = (current_pos[0] + direction[0], current_pos[1] + direction[1])
            if neighbour in visited:
                continue
            visited.add(neighbour)
            if neighbour in plots:
                continue
            neighbour_value = grid.get(neighbour)
            if neighbour_value != current_value:
                continue
            queue.append(neighbour)

    return plots

In [5]:
garden_plots = {}
all_visited_pos = {(-1, -1)}
for idx, (pos, value) in enumerate(grid.items()):
    if pos in all_visited_pos:
        continue
    plot_key = grid.get(pos) + "_" + str(idx)
    
    garden_plot_poses = bfs_find_paths(pos, idx)   
    garden_plots[plot_key] = garden_plot_poses
    
    all_visited_pos.update(garden_plot_poses)

In [6]:
def get_area(key):
    return len(garden_plots[key])

In [7]:
def get_perimeter(key):
    vals = garden_plots[key]
    perimeter = len(vals) * 4
    for a, b in combinations(vals, 2):
        x_diff = a[0] - b[0]
        y_diff = a[1] - b[1]
        if abs(x_diff) + abs(y_diff) == 1:
            perimeter -= 2
    return perimeter

In [8]:
def get_price(key):
    area = get_area(key)
    perimeter = get_perimeter(key)
    return area * perimeter

In [9]:
price = sum([get_price(k) for k in garden_plots.keys()])
print(f"Answer #1: {price}")

Answer #1: 1473276


# Part 2

In [10]:
dirs = [
        [[-0.5, 0], "left"],
        [[0.5, 0], "right"],
        [[0, -0.5], "up"],
        [[0, 0.5], "down"]
    ]

In [11]:
def get_num_corners(border_points):
    corners = 0
    invalid_up_down = {"up", "down"}
    invalid_left_right = {"left", "right"}

    alarm = 0
    for pos in border_points:
        borders = set()
        for d, name in dirs:
            new_coord = (pos[0] + d[0], pos[1] + d[1])
            if new_coord not in border_points:
                continue
            borders.add(name)
        
        if borders == invalid_up_down or borders == invalid_left_right:
            continue

        off_eight = [[-0.5, 0.5], [0.5, 0.5], [-0.5, -0.5], [0.5, -0.5]]
        for d in off_eight:
            new = (pos[0] + d[0], pos[1] + d[1])
            if new in border_points:
                alarm += 1
        corners += 1

    # Handle edge case when 3 are intersecting, shouldn't be the case.
    # Honestly, not a 100 how this works but science huh
    # Would likely break if same property on two sides
    # Might come back and fix more properly, likely some edge case in the off_eight/alarm calculation
    alarm_diff = alarm // 4
    if alarm_diff % 2 != 0:
        alarm_diff -= 1
    corners -= alarm_diff
    return corners

In [12]:
directions = [[-1, 0], [1, 0], [0, -1], [0, 1],[-1, -1], [-1, 1], [1, -1],[1, 1]]
def get_sides(key):
    vals = garden_plots[key]
    border = set()
    for val in vals:
        for d in directions:
            new_pos = (val[0] + d[0], val[1] + d[1])
            if new_pos in vals:
                continue
            new_pos_small = (val[0] + d[0] / 2, val[1] + d[1] / 2)
            border.add(new_pos_small)

    corners = get_num_corners(border)
    return border, corners


In [13]:
def get_price_part2(key):
    area = get_area(key)
    border, corners = get_sides(key)
    return area * corners
    

In [14]:
price = 0
for key in garden_plots.keys():
    price += get_price_part2(key)

print(f"Answer #2: {price}")

Answer #2: 901100
